# Tutorial 3

## Python Dictionaries

Dictionaries are data structures that store `key : value` pairs (if you want to learn more about how dictionaries are implemented, you may start here https://en.wikipedia.org/wiki/Hash_table). We start with the following example.

In [1]:
student1 = {
   "name": "Alice",
   "age": 21,
   "major": "Computer Science"
}
print(type(student1))

<class 'dict'>


So `student1` is a dictionary, and it has keys `name`, `age` and `major`. The key `name` maps to the value `Alice`, the key `age` maps to the value `21` and the key `major` maps to the value `Computer Science`. To access a value of the dictionary, we use its corresponding key.

In [2]:
print(student1["major"])

Computer Science


In [3]:
 print(student1[0])

KeyError: 0

The error above is because `0` is not a key of the dictionary `student1`. We can also create an empty dictionary and add `key : value` pairs to it.

In [4]:
student2 = {}
student2["name"] = "Bob"
student2["age"] = 20
student2["major"] = ["Mathematics", "Computer Science"]
print(student2)

{'name': 'Bob', 'age': 20, 'major': ['Mathematics', 'Computer Science']}


 Note that while `student1["major"]` is the string `"Computer Science"`, `student2["major"]` is the **list** `["Mathematics", "Computer Science"]`. We can use multiple types for the keys and the values of a dictionary. However, look at what happens below.

In [5]:
student2[["a", "b"]] = 1

TypeError: unhashable type: 'list'

So while we can use lists as **values**, we cannot use lists as **keys**. This is getting a bit more in-depth into the hash-tables that I mentioned previously, but essentially every object that we use as a **key** needs to be **hashable**, that is, it needs to implement a **hash function**. In general, in Python **immutable** objects are hashable, while **mutable** are not. As the name suggests, an immutable object is one that you cannot change its value. For instance, **tuples** are immutable variants of lists.

In [6]:
L = ["a", "b"] # this is a list
L[0] = "c"
T = ("a", "b") # this is a tuple
T[0] = "c"

TypeError: 'tuple' object does not support item assignment

The error above shows that while we can modify a list, we cannot modify a tuple. If you want, here is a reference for some extra reading on hash functions in Python: https://www.pythonmorsels.com/what-are-hashable-objects/ .
Given that tuples are immutable, we may now infer that tuples are hashable, while lists are not. So let's try to use a tuple as key to the dictionary `student2`.

In [7]:
student2[("a", "b")] = 1
print(student2)

{'name': 'Bob', 'age': 20, 'major': ['Mathematics', 'Computer Science'], ('a', 'b'): 1}


Great! So we learned that if we want to use a list as a key to some dictionary, then we first need to convert the list to a tuple. To end our discussion on dictionaries, let us now see how to iterate through the elements of a dictionary.

In [8]:
for key in student2:
    print(key)

name
age
major
('a', 'b')


And since for each key `key` we have a corresponding value `student2[key]`, we can access the `key : value` pairs of a dictionary as follows.

In [9]:
for key in student2:
    print("key: " + str(key) + ", value: " + str(student2[key]))

key: name, value: Bob
key: age, value: 20
key: major, value: ['Mathematics', 'Computer Science']
key: ('a', 'b'), value: 1


## Network Flow Gurobi Example

### Introduction
All the examples we’ve explored so far are “toy” problems designed to help you grasp the basics. Now, we will tackle a more complex integer programming problem that reflects challenges encountered in the real world.

Imagine we are **Operations Research analysts** for a major airline. Due to a pilot strike, many flights have been canceled, leaving numerous passengers dissatisfied—an issue that adversely affects our business. Fortunately, partner airlines have offered seats on their flights, giving us the opportunity to reallocate passengers from canceled flights. Our goal is to assign these passengers to available flights in a way that maximizes the number of individuals reaching their destinations.

More formally, let $c_1, c_2, \ldots, c_\ell$ represent the **canceled flights**. For each $i = 1, \ldots, \ell$, flight $c_i$ was originally scheduled to carry $p_i$ **passengers**. The partner airlines allow us to reallocate these passengers to **flights** $f_1, \ldots, f_k$, where each flight $f_i$ has $u_i$ **available seats**. We will model this problem as a "network-flow-type" problem (see Chapter 2.5 of the Lecture Notes) as follows.

We will represent the problem using a directed graph $G = (V, A)$. The set of nodes $V$ is given by $\{f_1, \ldots, f_k\} \cup \{s_i, t_i\}_{i = 1}^\ell$. For each $i = 1, \ldots, \ell$, vertex $s_i$ (respectively $t_i$) is an artificial source (respectively sink) vertex corresponding to the canceled flight $c_i$.

The set of arcs $A$ is constructed by identifying possible **connections** between the flights. Specifically, for every pair of flights $f_i$ and $f_j$ that can be connected (i.e., the destination of flight $f_i$ matches the origin of flight $f_j$, and flight $f_i$ arrives before flight $f_j$ departs), we include an arc $(f_i, f_j) \in A$. Additionally, we create arcs from each source $s_i$ to a flight $f_j$ if the passengers from the canceled flight $c_i$ can take flight $f_j$ (so the origin of $f_j$ is the same as the origin of $c_i$ and $f_j$ departs after the original departure time of $c_i$). Similarly, we also add arcs from a flight $f_j$ to a sink vertex $t_i$, if $f_j$ and $t_i$ have the same destination.

(**Remark:** This may not be the most "natural" way to model this problem. One might instead create vertices for each of the **airports** and model the flights as arcs. However, this approach fail to model the **flights connections**; for instance, with this approach, it is not clear how we would model the requirement that if flight $f_1$ occurs before flight $f_2$, then passengers cannot be assigned to flight $f_2$ before taking flight $f_1$. By creating nodes for each of the **flights**, we can model this constraint in the set of arcs $A$.)

### Mathematical formulation

For each $i = 1, \ldots, \ell$ and arc $uv \in A$, we have a **flow variable** $f^i_{uv}$, which is an integer variable and indicates the amount of passengers of flight $c_i$ that should travel through arc $uv$ (either making a connection between flights $u$ and $v$, or departuring/arriving in the origin/destination of flight $c_i$). We formulate this problem as follows.
\begin{align} 
\min &~~\sum_{i = 1}^\ell \sum_{a \in \delta^+(s_i)} f^i_a & \nonumber \\
\text{s.t.} &~~ \sum_{a \in \delta^-(v)} f^i_a - \sum_{a \in \delta^+(v)} f^i_a =
\begin{cases}
-p_i & \text{if } v = s_i, \\
p_i & \text{if } v = t_i, \\
0 & \text{otherwise,}
\end{cases} & \forall v \in V,~i = 1, \ldots, \ell, 
 \nonumber \\
&~~ \sum_{i = 1}^\ell \sum_{a \in \delta^-(f_j)} f^i_a \leq u_i, & \forall j = 1, \ldots, k, \nonumber \\ 
&~~ f^i_a \in \{0, \ldots, p_i\}, & \forall a \in A,~i = 1, \ldots, \ell. 
\end{align}

### Input file

The input to our problem will be given by a file that looks like this.
```
3 4
0 30
1 12
2 25
0 50
1 20
2 25
3 40
f0 f1
f0 f2
f1 f2
f1 f3
f2 f3
s0 f0
s1 f1
s2 f1
f2 t1
f3 t2
f3 t0
```

To simplify the coding we will do later, we start counting from 0 rather than 1. The first line of this file contain the integers $\ell$ and $k$. Then, each of the next $\ell$ lines contain the id $i$ and the number of passengers $p_i$. In this case $p_0 = 30, p_1 = 12$ and $p_2 = 25$. After these $\ell$ lines, we then have $k$ lines, each containing the id $j$ and the number of available seats $u_j$. So in this case, $u_0 = 50, u_1 = 20, u_2 = 25$ and $u_3 = 40$. Finally, the remaining lines of the file correspond to the arcs in our graph. For example, in this case, we have arcs $(f_0, f_1), (f_0, f_2), (f_1, f_2)$ and so on.

The image below illustrates the graph $G = (V, E)$ associated with this instance and a corresponding optimal solution. The number next to the nodes denote the number of passengers or the number of available seats. The numbers next to the dashed lines represent flow of passengers.

![image info](./graph.svg)

### Code

As usual, we start by setting up gurobi and reading the input file.

In [10]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("flights")

f = open("../data/flights.txt", "r")

data = f.readlines()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-16


We will use a variable `line_index` to iterate through the lines of the file. The first step is to read $\ell$ and $k$ from the first line.

In [11]:
# First line contains number of canceled flights and number of partner flights.
line_index = 0
L = data[0].strip().split(" ")
n_canceled = int(L[0])
n_flights = int(L[1])
line_index += 1
print("number of canceled flights: " + str(n_canceled))
print("number of partner flights: " + str(n_flights))

number of canceled flights: 3
number of partner flights: 4


Notice that we are calling `strip()`. This is so that we can get rid of the end-of-line characters `\n`. Check the next snippets.

In [12]:
data[0]

'3 4\n'

In [13]:
data[0].strip()

'3 4'

Now that we have $\ell$ and $k$, we construct the set of vertices $V$.

In [14]:
# Construct the set of vertices.
vertices = []
for i in range(n_canceled):
    vertices.append("s" + str(i))
    vertices.append("t" + str(i))
for j in range(n_flights):
    vertices.append("f" + str(j))

print(vertices)

['s0', 't0', 's1', 't1', 's2', 't2', 'f0', 'f1', 'f2', 'f3']


Next, we read the number of passengers in each cancelled flight.

In [15]:
# Now we have lines for each of the canceled flights with the number of passengers.
passengers = []
for i in range(n_canceled):
    L = data[line_index].strip().split(" ")
    line_index += 1
    passengers.append(int(L[1]))

print(passengers)

[30, 12, 25]


So the $i$-th entry of `passengers` is the value $p_i$. We can similarly get the number of available seats $u_j$, for $j = 1, \ldots, k$.

In [16]:
# Next, we have the lines with the partner flights and the number of seats.
seats = []
for i in range(n_flights):
    L = data[line_index].strip().split(" ")
    line_index += 1
    seats.append(int(L[1]))

print(seats)

[50, 20, 25, 40]


Now comes the part where we use **dictionaries**. We will represent the nodes in our graph with the strings we have in the vector `vertices`. The idea now is that, given a **key** $u \in V$, `out_neighbors[u]` will contain a list with all the vertices $v$ such that $(u, v) \in A$. Similarly, given a **key** $u \in V$, `in_neighbors[u]` will contain a list with all the vertices $v$ such that $(v, u) \in A$. The next snippet initializes these dictionaries with empty lists.

In [17]:
# To ease the implementation, we create dictionaries of out-neighbors and in-neighbors
# and we initialize these with empty lists.
out_neighbors = {}
in_neighbors = {}
for v in vertices:
    out_neighbors[v] = []
    in_neighbors[v] = []

We also create a dictionary to store the **Gurobi flow variables**.

In [18]:
# We also have a dictionary to store the gurobi flow variables.
flow = {}

Ok, let us now read the lines from our files that contain the **arcs**. While doing so, we will also populate the dictionaries we created before.

In [19]:
# Now we read the arcs.
while line_index < len(data):
    [u, v] = data[line_index].strip().split(" ")
    line_index += 1

    # Create the Gurobi variables.
    # Note that we use tuples as keys for the dictionary!
    for i in range(n_canceled):
        flow[(u, v, i)] = m.addVar(
            lb=0,
            ub=passengers[i],
            vtype=GRB.INTEGER,
            name="f_" + u + "," + v + "^" + str(i),
        )

    # Add entries to the in/out neighbor dictionaries.
    out_neighbors[u].append(v)
    in_neighbors[v].append(u)

print("out_neighbors: " + str(out_neighbors))
print("in_neighbors: " + str(in_neighbors))

out_neighbors: {'s0': ['f0'], 't0': [], 's1': ['f1'], 't1': [], 's2': ['f1'], 't2': [], 'f0': ['f1', 'f2'], 'f1': ['f2', 'f3'], 'f2': ['f3', 't1'], 'f3': ['t2', 't0']}
in_neighbors: {'s0': [], 't0': ['f3'], 's1': [], 't1': ['f2'], 's2': [], 't2': ['f3'], 'f0': ['s0'], 'f1': ['f0', 's1', 's2'], 'f2': ['f0', 'f1'], 'f3': ['f1', 'f2']}


And you should check with the previous figure that this seems to be ok. Now that we have the Gurobi variables and the data structures representing the graph (i.e., the dictionaries `out_neighbors` and `in_neighbors`), we are ready to write the code for the rest of our integer program.

In [20]:
# Set objective function.
obj_expr = 0
for u in vertices:
    if u[0] == "s":
        for v in out_neighbors[u]:
            i = int(u[1:])
            obj_expr += flow[(u, v, i)]

m.setObjective(obj_expr, GRB.MAXIMIZE)

# Set flow conservation constraints.
for i in range(n_canceled):
    for u in vertices:
        if u == "s" + str(i) or u == "t" + str(i):
            continue

        expr = 0
        for v in in_neighbors[u]:
            expr += flow[(v, u, i)]
        for v in out_neighbors[u]:
            expr -= flow[(u, v, i)]
        m.addConstr(expr == 0.0)

# Set seat capacity constraints.
for j in range(n_flights):
    expr = 0
    for v in in_neighbors["f" + str(j)]:
        for i in range(n_canceled):
            expr += flow[(v, "f" + str(j), i)]
    m.addConstr(expr <= seats[j])

Before we solve this model, I want to point out that, one way of debugging your model is to just print out the Gurobi expressions as follows. (Recall we call `update()` just so that we get a pretty printing.)

In [21]:
m.update()
print(obj_expr)

f_s0,f0^0 + f_s1,f1^1 + f_s2,f1^2


However, an alternative that might be better is to just ask Gurobi to write the model as an `.lp` file. This can be done easily.

In [22]:
m.write("test.lp")

And I suggest you to take a look at this file. We will close this tutorial solving the model and getting an optimal solution.

In [23]:
m.optimize()

if m.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print("Objective value:", m.objVal)
    for v in m.getVars():
        if v.x != 0:
            print(v.varName, "=", v.x)
            
else:
    print("Error: Optimal solution was not found.")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-4720HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 33 columns and 84 nonzeros
Model fingerprint: 0x1974f8c4
Variable types: 0 continuous, 33 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 3e+01]
  RHS range        [2e+01, 5e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 25 rows and 30 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 6 nonzeros
Found heuristic solution: objective 40.0000000
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 4.250000e+01, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No